In [225]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import quadrature
from math import sinh, factorial

In [226]:
def K(x, y):
    return sinh(x * y)


def f(x):
    return 2 - x + x**2

def get_values(u):
    h = (b - a) / N
    values = np.zeros((N + 1, 2))
    for k in range(N + 1):
        values[k, 0] = a+k*h
        values[k, 1] = u(a+k*h)

    return values

def norm(res1, res2):
    return max([abs(res2[i][1] - res1[i][1]) for i in range(N + 1)])


In [227]:

a, b = 0, 1
N = 10

gauss_nodes_4n_xk = [-0.8611363116, -0.3399810436, 0.3399810436, 0.8611363116]
gauss_nodes_4n_ak = [0.3478548451, 0.6521451549, 0.6521451549, 0.3478548451]
gauss_nodes_5n_xk = [-0.9061798459, -0.5384693101, 0, 0.5384693101, 0.9061798459]
gauss_nodes_5n_ak = [0.2369268851, 0.4786286705, 0.5688888889, 0.4786286705, 0.2369268851]

for i in range(4):
    gauss_nodes_4n_xk[i] = (b-a)/2*gauss_nodes_4n_xk[i]+(b+a)/2
    gauss_nodes_4n_ak[i] = (b-a)/2*gauss_nodes_4n_ak[i]

for i in range(5):
    gauss_nodes_5n_xk[i] = (b-a)/2*gauss_nodes_5n_xk[i]+(b+a)/2
    gauss_nodes_5n_ak[i] = (b-a)/2*gauss_nodes_5n_ak[i]

def solve_MMK(n, gauss_nodes_xk, gauss_nodes_ak):
    D = np.zeros((n, n))
    g = np.zeros(n)

    for i in range(n):
        for j in range(n):
            D[i][j] = (i == j) + gauss_nodes_ak[j] * K(gauss_nodes_xk[i], gauss_nodes_xk[j])

    for i in range(n):
        g[i] = f(gauss_nodes_xk[i])

    z = np.linalg.solve(D, g)
    return z


def values(x_vals, u):
    return [u(x) for x in x_vals]

z_4 = solve_MMK(4, gauss_nodes_4n_xk, gauss_nodes_4n_ak)
u_new_4 = lambda x: f(x) - gauss_nodes_4n_ak[0] * K(x, gauss_nodes_4n_xk[0]) * z_4[0] - gauss_nodes_4n_ak[1] * K(x, gauss_nodes_4n_xk[1]) * z_4[1] - gauss_nodes_4n_ak[2] * K(x, gauss_nodes_4n_xk[2]) * z_4[2] - gauss_nodes_4n_ak[3] * K(x, gauss_nodes_4n_xk[3]) * z_4[3]

val_4 = get_values(u_new_4)



z_5 = solve_MMK(5, gauss_nodes_5n_xk, gauss_nodes_5n_ak)
u_new_5 = lambda x: f(x) - gauss_nodes_5n_ak[0] * K(x, gauss_nodes_5n_xk[0]) * z_5[0] - gauss_nodes_5n_ak[1] * K(x, gauss_nodes_5n_xk[1]) * z_5[1] - gauss_nodes_5n_ak[2] * K(x, gauss_nodes_5n_xk[2]) * z_5[2] - gauss_nodes_5n_ak[3] * K(x, gauss_nodes_5n_xk[3]) * z_5[3] - gauss_nodes_5n_ak[4] * K(x, gauss_nodes_5n_xk[4]) * z_5[4]
val_5 = get_values(u_new_5)

delta = norm(val_5, val_4)

x = np.arange(-100, 100, 1)

In [228]:
val_4

array([[0.        , 2.        ],
       [0.1       , 1.84202271],
       [0.2       , 1.70372195],
       [0.3       , 1.58477215],
       [0.4       , 1.48484342],
       [0.5       , 1.40359947],
       [0.6       , 1.3406953 ],
       [0.7       , 1.29577497],
       [0.8       , 1.26846921],
       [0.9       , 1.25839292],
       [1.        , 1.26514262]])

In [229]:
val_5

array([[0.        , 2.        ],
       [0.1       , 1.84202271],
       [0.2       , 1.70372195],
       [0.3       , 1.58477215],
       [0.4       , 1.48484342],
       [0.5       , 1.40359947],
       [0.6       , 1.3406953 ],
       [0.7       , 1.29577498],
       [0.8       , 1.26846921],
       [0.9       , 1.25839292],
       [1.        , 1.26514263]])

In [230]:
delta

1.0709157782784473e-08

In [231]:
def alpha(k):
    return lambda x: x**(2*k + 1)

def beta(k):
    return lambda y: y**(2*k + 1)/factorial(2*k + 1)

def scalar(y, z):
    result, error = quadrature(lambda x: y(x) * z(x), a, b)
    return result

def solve_change(n):
    G = np.zeros((n, n))
    F = np.zeros((n, 1))

    for j in range(n):
        for i in range(n):
            G[j][i] = (i == j) + scalar(beta(j), alpha(i))

    for j in range(n):
        F[j] = scalar(beta(j), lambda x: f(x))

    A = np.linalg.solve(G, F)

    def U(x):
        return f(x) - sum([A[i] * alpha(i)(x) for i in range(n)])

    return U

delta_core = sum([alpha(i)(0.5) * beta(i)(0.5) for i in range(4)]) - K(0.5, 0.5)

u_3 = solve_change(5)
u_4 = solve_change(6)

val_change_3 = get_values(u_3)
val_change_4 = get_values(u_4)

delta_change = norm(val_change_3, val_change_4)

In [232]:
delta_core

-1.0518252935298733e-11

K(x, y)

In [233]:
val_change_3

array([[0.        , 2.        ],
       [0.1       , 1.84202271],
       [0.2       , 1.70372195],
       [0.3       , 1.58477215],
       [0.4       , 1.48484342],
       [0.5       , 1.40359947],
       [0.6       , 1.3406953 ],
       [0.7       , 1.29577498],
       [0.8       , 1.26846921],
       [0.9       , 1.25839292],
       [1.        , 1.26514264]])

In [234]:
val_change_4

array([[0.        , 2.        ],
       [0.1       , 1.84202271],
       [0.2       , 1.70372195],
       [0.3       , 1.58477215],
       [0.4       , 1.48484342],
       [0.5       , 1.40359947],
       [0.6       , 1.3406953 ],
       [0.7       , 1.29577498],
       [0.8       , 1.26846921],
       [0.9       , 1.25839292],
       [1.        , 1.26514264]])

In [235]:
delta_change

1.0919567472456038e-09